In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace = ws, name = "udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-257061
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-257061


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C": choice(.001, .01, .1, 1, 10, 100),
        "--max_iter": choice(10, 20, 50, 100, 200, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = .1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = '.',
                      script = 'train.py',
                      compute_target = compute_target,
                      environment = sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = src, 
                                     hyperparameter_sampling = ps, 
                                     policy = policy,
                                     primary_metric_name = 'Accuracy', 
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs = 4, 
                                     max_concurrent_runs = 4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(hyperdrive_config, show_output = True)

In [5]:
RunDetails(run).show()
run.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_996a0844-c84e-47e3-a742-331eb92544e7
Web View: https://ml.azure.com/runs/HD_996a0844-c84e-47e3-a742-331eb92544e7?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-257061/workspaces/quick-starts-ws-257061&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-04-05T23:25:27.094269][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-04-05T23:25:27.5231125Z][SCHEDULER][INFO]Scheduling job, id='HD_996a0844-c84e-47e3-a742-331eb92544e7_0' 
[2024-04-05T23:25:27.6577558Z][SCHEDULER][INFO]Scheduling job, id='HD_996a0844-c84e-47e3-a742-331eb92544e7_1' 
[2024-04-05T23:25:27.7606315Z][SCHEDULER][INFO]Scheduling job, id='HD_996a0844-c84e-47e3-a742-331eb92544e7_2' 
[2024-04-05T23:25:27.7782104Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_996a0844-c84e-47e3-a742-331eb92544e7_0' 
[2024-04-05T23:25:27.8825277Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_996a0844-c84e-47e3-a742-

{'runId': 'HD_996a0844-c84e-47e3-a742-331eb92544e7',
 'target': 'cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2024-04-05T23:25:26.375627Z',
 'endTimeUtc': '2024-04-05T23:30:29.35224Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '96f7bed2-4348-4e8e-81bc-81bf8d21c5b0',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '36',
  'best_child_run_id': 'HD_996a0844-c84e-47e3-a742-331eb92544e7_3',
  'score': '0.9094440398154892',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_996a0844-c84e-47e3-a742-331eb92544e7_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType

In [6]:
import joblib

# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()

In [7]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-04-05-23',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [8]:
model = best_run.register_model(model_name = 'best_run_hyperdrive', model_path = 'outputs/model.joblib')

In [9]:
# The best run's hyperparameter set
import pandas as pd

parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
pd.Series(best_parameters, name = 'Value').to_frame()

,Value
--C,0.1
--max_iter,50


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [11]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)
y_train = pd.DataFrame(y_train, columns = ['y'])
training_data = pd.concat([x_train, y_train], axis = 1)
training_data.to_csv('./training/training_data.csv')

datastore = ws.get_default_datastore()
datastore.upload(src_dir = './training', target_path = './training')
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ('./training/training_data.csv'))])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Target already exists. Skipping upload for training/training_data.csv
Uploaded 0 files


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=3)

In [13]:
# Submit your automl run
run = exp.submit(config = automl_config, show_output = True)
run.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_173d9a99-d35c-4dce-a754-5cbda78ee4c2,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2024/04/05 23:41:14 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2756                          |1                               |24712                                 |
+------------------------------+--------------------------------+-----

2024-04-06:00:05:35,906 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


{'runId': 'AutoML_173d9a99-d35c-4dce-a754-5cbda78ee4c2',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-04-05T23:31:48.926418Z',
 'endTimeUtc': '2024-04-06T00:04:18.681396Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-257061","workspace_name":"quick-starts-ws-257061","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,

In [16]:
# Retrieve and save your best automl model.
best_run, fitted_model = run.get_output()
best_run.register_model(model_name = 'best_run_automl.joblib', model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-257061', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55', resource_group='aml-quickstarts-257061'), name=best_run_automl.joblib, id=best_run_automl.joblib:1, version=1, tags={}, properties={})